In [1]:
"""
Author: Ryan Han
Date Updated: 5/7/2020
Description: This .ipynb notebook asks for a player's full name, team, and a specific year.
Once inputted, the script will access NBA game statistics through 'nba_api',API Client for www.nba.com
and produce a detailed Pandas dataframe of regular season field goal attempts by the specified player during the specified NBA year.
Input: NBA player's full name, current team, and NBA season start year
Output: Pandas DataFrame
"""


# Modules
from nba_api.stats.static import players
from nba_api.stats.static import teams
from nba_api.stats.endpoints import shotchartdetail as scd


# Functions
# Input 'fullname' argument as a string of player's full name
def player_info(fullname):
    player_dict = players.get_players()
    for player in player_dict:
        if player['full_name'] == fullname:
            return(player)


# Input 'teamname' argument as a string of full team name
def team_info(teamname):
    nba_teams = teams.get_teams()
    for team in nba_teams:
        if team['full_name'] == teamname:
            return(team)


def clean_df(df):
    df = df.drop_duplicates()
    df.dropna(inplace=True)
    clean_df = df[['PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_NAME', 'SHOT_ZONE_BASIC',
                   'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_DISTANCE',
                   'LOC_X', 'LOC_Y', 'SHOT_MADE_FLAG', 'GAME_DATE']]
    clean_df = clean_df.rename(columns={'PLAYER_ID': 'id', 'PLAYER_NAME': 'name', 'TEAM_ID': 'team_id', 'TEAM_NAME': 'team_name',
                                        'SHOT_ZONE_BASIC': 'shot_zone_basic', 'SHOT_ZONE_AREA': 'shot_zone_area',
                                        'SHOT_ZONE_RANGE': 'shot_zone_range', 'SHOT_DISTANCE': 'shot_distance',
                                        'LOC_X': 'x', 'LOC_Y': 'y', 'SHOT_MADE_FLAG': 'shot_made', 'GAME_DATE': 'game_date'})
    return(clean_df)

# function to separate dataframe by years
# Since nba seasons start in one year and end in the next, had to include 'year + 1'
# The NBA regular season usually starts in October and ends by May


def separate_year_charts(year):
    chart_by_year = shotchart_df[(shotchart_df['game_date'] > (str(year) + '1001')) &
                                 (shotchart_df['game_date'] < (str(year + 1) + '0531'))]
    return(chart_by_year)


# inputs to run functions
fullname = input("Please input the full name of the player: ")
teamname = input("Please input the full name of this player's current team: ")
year = input("Please input start year for NBA season: ")

# assign variables to return values
player = player_info(fullname.title())
team = team_info(teamname.title())

stats = scd.ShotChartDetail(player_id=player['id'],
                            team_id=team['id'], context_measure_simple="FGA")
shots_df = stats.get_data_frames()[0]

shotchart_df = clean_df(shots_df)
shotchart = separate_year_charts(int(year))
shotchart

Please input the full name of the player:  Giannis Antetokounmpo
Please input the full name of this player's current team:  Milwaukee Bucks
Please input start year for NBA season:  2016


,id,name,team_id,team_name,shot_zone_basic,shot_zone_area,shot_zone_range,shot_distance,x,y,shot_made,game_date
2209,203507,Giannis Antetokounmpo,1610612749,Milwaukee Bucks,Restricted Area,Center(C),Less Than 8 ft.,1,0,16,0,20161026
2210,203507,Giannis Antetokounmpo,1610612749,Milwaukee Bucks,Mid-Range,Center(C),8-16 ft.,15,68,139,0,20161026
2211,203507,Giannis Antetokounmpo,1610612749,Milwaukee Bucks,Restricted Area,Center(C),Less Than 8 ft.,2,-27,-1,0,20161026
2212,203507,Giannis Antetokounmpo,1610612749,Milwaukee Bucks,Restricted Area,Center(C),Less Than 8 ft.,0,0,1,1,20161026
2213,203507,Giannis Antetokounmpo,1610612749,Milwaukee Bucks,Restricted Area,Center(C),Less Than 8 ft.,0,0,1,1,20161026
...,...,...,...,...,...,...,...,...,...,...,...,...
3462,203507,Giannis Antetokounmpo,1610612749,Milwaukee Bucks,In The Paint (Non-RA),Center(C),Less Than 8 ft.,5,-52,16,0,20170410
3463,203507,Giannis Antetokounmpo,1610612749,Milwaukee Bucks,Restricted Area,Center(C),Less Than 8 ft.,0,-6,2,1,20170410
3464,203507,Giannis Antetokounmpo,1610612749,Milwaukee Bucks,Restricted Area,Center(C),Less Than 8 ft.,1,-11,13,0,20170410
3465,203507,Giannis Antetokounmpo,1610612749,Milwaukee Bucks,Mid-Range,Right Side Center(RC),16-24 ft.,18,130,129,0,20170410
